# COGS 108 - EDA Checkpoint

# Names

- Jianfan Huo
- Yunhao Du
- Yixin Jiang
- Yixiao An
- Celine Zhao

<a id='research_question'></a>
# Research Question

Is there a positive relationship between state economic development and vaccine coverage？

# Setup

In [1]:
## YOUR CODE HERE
## YOUR CODE HERE
# import dataframes
import pandas as pd

# import number manipulation
import numpy as np 

# import data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#import Statmodels & patsy
import patsy

In [2]:
df_vaccine = pd.read_csv('Data/COVID-19_Vaccinations_in_the_United_States_Jurisdiction.csv')
df_population = pd.read_csv('Data/Population.csv')
df_income = pd.read_csv('Data/net_income.csv')
df_GDP = pd.read_csv('Data/states_GDP.csv')

# Data Cleaning

Describe your data cleaning steps here.
## Cleaning #1
Our goal was to analyze whether state economies contributed to the spread of vaccines. The data we need here is the number of vaccinations in the past year for each region. So, the data we need here is Date, Location, Series_Complete_Yes, Series_Complete_Janssen, Series_Complete_Moderna, Series_Complete_Pfizer, and	Series_Complete_Unk_Manuf

In [3]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
df_vaccine = df_vaccine[['Date', 
                         'Location', 
                         'Series_Complete_Yes', 
                         'Series_Complete_Janssen', 
                         'Series_Complete_Moderna',
                         'Series_Complete_Pfizer',
                         'Series_Complete_Unk_Manuf']]

The number of all State/Territory/Federal Entity in US is 65.

In [4]:
df_vaccine.head(64)

,Date,Location,Series_Complete_Yes,Series_Complete_Janssen,Series_Complete_Moderna,Series_Complete_Pfizer,Series_Complete_Unk_Manuf
0,11/05/2021,SC,2587217,185786,957836,1443294,301
1,11/05/2021,OR,2661407,228418,950422,1481340,1227
2,11/05/2021,IN,3365638,259526,1196470,1902099,7543
3,11/05/2021,NY,13038415,1127875,4619753,7287968,2819
4,11/05/2021,WI,3409332,276848,1264326,1867685,473
...,...,...,...,...,...,...,...
59,11/05/2021,CO,3564615,279662,1338511,1944748,1694
60,11/05/2021,AL,2207077,146027,939883,1120694,473
61,11/05/2021,VI,49413,1483,13107,34819,4
62,11/05/2021,MS,1369214,75803,543059,749820,532


## Cleaning #2
Current official state personal income data are only available through the first two quarters of 2021, so we need to filter our data again. Because universal vaccination began in March, it didn't widely spread until April. So we decided to use data from the second quarter(April-June) to analyze the impact of each state's economy on vaccination penetration. In this step, we delete the data of dates, and only keep the 'Date' from "4/01/2021" to "6/30/2021".

In [5]:
# Check the type of each data in the dataset 
df_vaccine.dtypes

Date                         object
Location                     object
Series_Complete_Yes           int64
Series_Complete_Janssen       int64
Series_Complete_Moderna       int64
Series_Complete_Pfizer        int64
Series_Complete_Unk_Manuf     int64
dtype: object

In [6]:
start = "04/01/2021"
end = "06/30/2021"

df_vaccine = df_vaccine[(df_vaccine['Date'][:] >= start ) & (df_vaccine['Date'][:] <= end)]

In [7]:
df_vaccine.reset_index(drop=True)
df_vaccine

,Date,Location,Series_Complete_Yes,Series_Complete_Janssen,Series_Complete_Moderna,Series_Complete_Pfizer,Series_Complete_Unk_Manuf
8305,06/30/2021,ME,819500,103312,327721,388364,103
8306,06/30/2021,UT,1196468,119014,456194,621258,2
8307,06/30/2021,PA,6345433,537395,2503367,3304347,324
8308,06/30/2021,GU,89574,4314,32664,52596,0
8309,06/30/2021,NY,10508196,894628,4043410,5569076,1082
...,...,...,...,...,...,...,...
14215,04/01/2021,NH,247295,16199,129723,101367,6
14216,04/01/2021,KY,798150,45857,358920,393257,116
14217,04/01/2021,NY,3362056,214142,1434622,1713049,243
14218,04/01/2021,SC,792963,49667,267357,475736,203


## Cleaning #3
Change the unit of data in df_vaccine to thousands and round the value up. 
Renaming all unclear columns, sorting all data by date and location, and only keeping the data for 51 states.

In [8]:
# Renaming all unclear columns
df_vaccine = df_vaccine.rename(columns = {'Series_Complete_Yes':'Num of Vaccinated People in thousand',
                        'Series_Complete_Janssen':'Num of Vaccinated People with Janssen in thousand',
                         'Series_Complete_Moderna':'Num of Vaccinated People with Moderna in thousand',
                         'Series_Complete_Pfizer':'Num of Vaccinated People with Pfizer in thousand',
                         'Series_Complete_Unk_Manuf':'Num of Vaccinated People with Unknown Manuf'
                        })
df_vaccine

,Date,Location,Num of Vaccinated People in thousand,Num of Vaccinated People with Janssen in thousand,Num of Vaccinated People with Moderna in thousand,Num of Vaccinated People with Pfizer in thousand,Num of Vaccinated People with Unknown Manuf
8305,06/30/2021,ME,819500,103312,327721,388364,103
8306,06/30/2021,UT,1196468,119014,456194,621258,2
8307,06/30/2021,PA,6345433,537395,2503367,3304347,324
8308,06/30/2021,GU,89574,4314,32664,52596,0
8309,06/30/2021,NY,10508196,894628,4043410,5569076,1082
...,...,...,...,...,...,...,...
14215,04/01/2021,NH,247295,16199,129723,101367,6
14216,04/01/2021,KY,798150,45857,358920,393257,116
14217,04/01/2021,NY,3362056,214142,1434622,1713049,243
14218,04/01/2021,SC,792963,49667,267357,475736,203


In [9]:
# Sorting all data by date and location
df_vaccine = df_vaccine.sort_values(["Date", "Location"], ascending = (True, True))
df_vaccine = df_vaccine.reset_index(drop=True)
df_vaccine

,Date,Location,Num of Vaccinated People in thousand,Num of Vaccinated People with Janssen in thousand,Num of Vaccinated People with Moderna in thousand,Num of Vaccinated People with Pfizer in thousand,Num of Vaccinated People with Unknown Manuf
0,04/01/2021,AK,165325,5693,71943,87679,10
1,04/01/2021,AL,673773,42958,314533,316247,35
2,04/01/2021,AR,432735,27498,204413,200767,57
3,04/01/2021,AS,13406,4,2775,10627,0
4,04/01/2021,AZ,1267484,48303,543865,674790,526
...,...,...,...,...,...,...,...
5910,06/30/2021,VT,409254,37291,161597,210149,217
5911,06/30/2021,WA,4150522,346036,1604907,2197981,1598
5912,06/30/2021,WI,2879461,227599,1140567,1511006,289
5913,06/30/2021,WV,666545,35141,299057,332152,195


In [10]:
# Only keeping the data for 51 states
states = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID',
          'IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO',
          'MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA',
          'RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
df_vaccine = df_vaccine[(df_vaccine['Location'][:].isin(states))]
df_vaccine = df_vaccine.reset_index(drop = True)
df_vaccine.head()

,Date,Location,Num of Vaccinated People in thousand,Num of Vaccinated People with Janssen in thousand,Num of Vaccinated People with Moderna in thousand,Num of Vaccinated People with Pfizer in thousand,Num of Vaccinated People with Unknown Manuf
0,04/01/2021,AK,165325,5693,71943,87679,10
1,04/01/2021,AL,673773,42958,314533,316247,35
2,04/01/2021,AR,432735,27498,204413,200767,57
3,04/01/2021,AZ,1267484,48303,543865,674790,526
4,04/01/2021,CA,6485628,368034,2823850,3293464,280


In [11]:
df_vaccine['Num of Vaccinated People in thousand'] = (df_vaccine['Num of Vaccinated People in thousand'] / 1000).round(2)
df_vaccine['Num of Vaccinated People with Janssen in thousand'] = (df_vaccine['Num of Vaccinated People with Janssen in thousand']/1000).round(2)
df_vaccine['Num of Vaccinated People with Moderna in thousand'] = (df_vaccine['Num of Vaccinated People with Moderna in thousand']/1000).round(2)
df_vaccine['Num of Vaccinated People with Pfizer in thousand'] = (df_vaccine['Num of Vaccinated People with Pfizer in thousand']/1000).round(2)
df_vaccine

,Date,Location,Num of Vaccinated People in thousand,Num of Vaccinated People with Janssen in thousand,Num of Vaccinated People with Moderna in thousand,Num of Vaccinated People with Pfizer in thousand,Num of Vaccinated People with Unknown Manuf
0,04/01/2021,AK,165.32,5.69,71.94,87.68,10
1,04/01/2021,AL,673.77,42.96,314.53,316.25,35
2,04/01/2021,AR,432.74,27.50,204.41,200.77,57
3,04/01/2021,AZ,1267.48,48.30,543.86,674.79,526
4,04/01/2021,CA,6485.63,368.03,2823.85,3293.46,280
...,...,...,...,...,...,...,...
4636,06/30/2021,VT,409.25,37.29,161.60,210.15,217
4637,06/30/2021,WA,4150.52,346.04,1604.91,2197.98,1598
4638,06/30/2021,WI,2879.46,227.60,1140.57,1511.01,289
4639,06/30/2021,WV,666.54,35.14,299.06,332.15,195


## Cleaning #4
We will get population data and remove excess data.

In [12]:
df_population.head(5)

,rank,State,Pop,Growth,Pop2018,Pop2010,growthSince2010,Percent,density
0,1,California,39613493,0.0038,39461588,37319502,0.0615,0.1184,254.2929
1,2,Texas,29730311,0.0385,28628666,25241971,0.1778,0.0889,113.8081
2,3,Florida,21944577,0.0330,21244317,18845537,0.1644,0.0656,409.2229
3,4,New York,19299981,-0.0118,19530351,19399878,-0.0051,0.0577,409.5400
4,5,Pennsylvania,12804123,0.0003,12800922,12711160,0.0073,0.0383,286.1704


In [13]:
df_population = df_population.drop(30)
df_population = df_population.iloc[: , 1:3]

In [14]:
df_population = df_population.sort_values(['State'], ascending = True)
df_population = df_population.reset_index(drop = True)

In [15]:
df_population['State'] = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID',
              'IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
              'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC',
              'SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

In [16]:
df_population = df_population.rename(columns = {'Pop':'Population'})
df_population.head()

,State,Population
0,AL,4934193
1,AK,724357
2,AZ,7520103
3,AR,3033946
4,CA,39613493


## Cleaning #5
In this step, we need to rearrange personal income and GDP data for each state. We deleted the geographic information (GeoFips) for each state. Since our data only analyzed personal income and GDP data for the second quarter of 2021, we also removed personal income and GDP data for the first quarter of 2021.

In [17]:
df_income.head(5)

,GeoFips,GeoName,2021:Q1,2021:Q2
0,0,United States,66191,62215
1,1000,Alabama,52633,48133
2,2000,Alaska *,69683,65501
3,4000,Arizona,54620,50373
4,5000,Arkansas,53643,49079


In [18]:
df_income = df_income[['GeoName','2021:Q2']]
df_income.columns = ['State', 'Personal_Income']

In [19]:
df_income = df_income.iloc[:52 , :]

In [20]:
df_income['State'] = ['U.S','AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID',
              'IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
              'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC',
              'SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

In [21]:
df_income.head(5)

,State,Personal_Income
0,U.S,62215
1,AL,48133
2,AK,65501
3,AZ,50373
4,AR,49079


## Cleaning #6
Abbreviate the full name of each state(include the DC) and drop the dataset which is not the state.


In [22]:
df_GDP.head(5)

,GeoFips,GeoName,2021:Q1,2021:Q2
0,0,United States *,22038226.0,22740959.0
1,1000,Alabama,237303.0,243554.6
2,2000,Alaska,52373.8,54020.3
3,4000,Arizona,388598.8,400155.7
4,5000,Arkansas,139172.7,143438.4


In [23]:
df_GDP = df_GDP[['GeoName','2021:Q2']]
df_GDP.columns = ['State', 'GDP']

In [24]:
df_GDP = df_GDP.iloc[:52 , :]

In [25]:
df_GDP['State'] = ['U.S','AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID',
              'IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
              'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC',
              'SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

In [26]:
df_GDP.head(5)

,State,GDP
0,U.S,22740959.0
1,AL,243554.6
2,AK,54020.3
3,AZ,400155.7
4,AR,143438.4


In [27]:
# Combine the df_GDP and df_income together
df_income['GDP'] = df_GDP['GDP']
df_econ = df_income
df_econ.head()

,State,Personal_Income,GDP
0,U.S,62215,22740959.0
1,AL,48133,243554.6
2,AK,65501,54020.3
3,AZ,50373,400155.7
4,AR,49079,143438.4


## Cleaning #7
Combine the df_population and df_econ together.
Change the unit of data in df_econ and round the value up. 

In [28]:
population = df_population['Population']
sum_population = sum(population)
sum_series = pd.Series([sum_population])
population = sum_series.append(population)
population = population.reset_index(drop = True)
df_econ["Population"] = population

In [29]:
df_econ['Personal_Income'] = (df_econ['Personal_Income']/1000).round(2)
df_econ['GDP'] = (df_econ['GDP']/1000).round(2)
df_econ['Population'] = (df_econ['Population']/1000).round(2)

In [30]:
df_econ = df_econ.rename(columns = {'Personal_Income':'Personal_Income in thousand',
                        'GDP':'GDP in thousand',
                         'Population':'Populationn in thousand'
                        })

### Finally, we get two clear and clean data frames which are df_vaccine, df_econ

In [31]:
df_vaccine

,Date,Location,Num of Vaccinated People in thousand,Num of Vaccinated People with Janssen in thousand,Num of Vaccinated People with Moderna in thousand,Num of Vaccinated People with Pfizer in thousand,Num of Vaccinated People with Unknown Manuf
0,04/01/2021,AK,165.32,5.69,71.94,87.68,10
1,04/01/2021,AL,673.77,42.96,314.53,316.25,35
2,04/01/2021,AR,432.74,27.50,204.41,200.77,57
3,04/01/2021,AZ,1267.48,48.30,543.86,674.79,526
4,04/01/2021,CA,6485.63,368.03,2823.85,3293.46,280
...,...,...,...,...,...,...,...
4636,06/30/2021,VT,409.25,37.29,161.60,210.15,217
4637,06/30/2021,WA,4150.52,346.04,1604.91,2197.98,1598
4638,06/30/2021,WI,2879.46,227.60,1140.57,1511.01,289
4639,06/30/2021,WV,666.54,35.14,299.06,332.15,195


In [32]:
df_econ

,State,Personal_Income in thousand,GDP in thousand,Populationn in thousand
0,U.S,62.22,22740.96,331343.57
1,AL,48.13,243.55,4934.19
2,AK,65.50,54.02,724.36
3,AZ,50.37,400.16,7520.10
4,AR,49.08,143.44,3033.95
5,CA,74.30,3290.17,39613.49
6,CO,66.68,416.94,5893.63
7,CT,81.85,294.65,3552.82
8,DE,57.50,79.28,990.33
9,DC,90.04,151.39,714.15


# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [33]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
